In [1]:
import numpy as np 
import pandas as pd 
from sklearn.datasets import load_iris
from sklearn.model_selection import train_test_split
import nbimporter
from ML_classes import LinearRegression

In [2]:
iris = load_iris(as_frame=True)
df = iris.data
df["target"] = iris.target

#Binary Classification 
df = df[(df["target"] == 0) | (df["target"] == 1)]
df.head()

,sepal length (cm),sepal width (cm),petal length (cm),petal width (cm),target
0,5.1,3.5,1.4,0.2,0
1,4.9,3.0,1.4,0.2,0
2,4.7,3.2,1.3,0.2,0
3,4.6,3.1,1.5,0.2,0
4,5.0,3.6,1.4,0.2,0


In [40]:
class LogisticRegression(LinearRegression):
    def __init__(self,data,bias=1,alpha=0.01,target="target"):
        self.data = data
        self.target_name = target
        self.target = np.array(data[target])
        self.data = data.drop(columns=target)
        self.data = self.data.reset_index(drop=True)
        self.bias = bias 
        self.alpha = alpha 
        self.theta_vector = np.zeros(self.data.shape[1] + 1)
        self.m = len(self.data)
    def Tx(self,sample_n=None):
        x_features = self.get_xfeatures_ytarget(sample_n=sample_n)[0]
        Tx = np.dot(self.theta_vector,x_features)
        return Tx 
    def activation(self,Tx):
        return 1 / (1 + np.exp(-1*Tx))
    def get_xfeatures_ytarget(self, sample_n):
        x_features = np.insert(self.data.iloc[sample_n,:],0,self.bias)
        y_target = self.target[sample_n]
        return (x_features,y_target)
    def J(self,sample_n):
        x_features,y_i = self.get_xfeatures_ytarget(sample_n=sample_n)
        h = self.activation(self.Tx(sample_n=sample_n))  # Sigmoid of Tx
        J = -(y_i * np.log(h) + (1 - y_i) * np.log(1 - h))
        return J 
    def gradient_descent(self,s1):
        x_features,y_i = self.get_xfeatures_ytarget(sample_n=s1)
        if y_i == 1:
            gradient = -(1 - self.activation(self.Tx(sample_n=s1))) * x_features
        if y_i == 0:
            gradient = self.activation(self.Tx(sample_n=s1)) * x_features
        self.gradient = gradient 
        return gradient
    def update_T(self):
        self.theta_vector -= (self.alpha * self.gradient)
    def iterations(self,n=100):
        cost_dict = {}
        iterations = 0 
        for i in range(n):
            total_cost = 0
            iterations +=1
            for index,sample in self.data.iterrows():
                total_cost += self.J(sample_n=index)
                self.gradient_descent(s1=index)
                self.update_T()
            cost_mean = (total_cost / self.m)
            cost_dict[i] = (cost_mean, self.theta_vector.copy())
            
        return cost_dict        

In [4]:
test = df.iloc[-3:,:]
df = df.iloc[:-3,:]

test

,sepal length (cm),sepal width (cm),petal length (cm),petal width (cm),target
97,6.2,2.9,4.3,1.3,1
98,5.1,2.5,3.0,1.1,1
99,5.7,2.8,4.1,1.3,1


In [5]:
LR = LogisticRegression(data=df)

In [6]:
x = LR.iterations(n=100)

In [7]:
cost,parameters = x.get(99)

In [8]:
bias = parameters[0]
parameters = parameters[1:]
print(bias)
print(parameters)

-0.3776946878192275
[-0.53362547 -1.96631818  3.00230506  1.30739529]


In [9]:
test["linear_combination"] = test.iloc[:,:-1].apply(lambda x: np.dot(x,parameters)+ bias,axis=1)
test["predicted"] = test["linear_combination"].apply(lambda x: (1 / 1 + np.exp(-x)))

In [12]:
test

,sepal length (cm),sepal width (cm),petal length (cm),petal width (cm),target,linear_combination,predicted
97,6.2,2.9,4.3,1.3,1,5.221030,1.005402
98,5.1,2.5,3.0,1.1,1,2.430070,1.088031
99,5.7,2.8,4.1,1.3,1,5.084014,1.006195


In [55]:
iris = load_iris(as_frame=True)


X_train,X_test,y_train,y_test = train_test_split(iris.data,iris.target,train_size=0.8,test_size=0.2,stratify=iris.target)



iris_train = pd.concat([X_train,pd.DataFrame(y_train)],axis=1)
iris_train = iris_train[(iris_train["target"] == 0)| (iris_train["target"] == 1)]
iris_test= pd.concat([X_test,pd.DataFrame(y_test)],axis=1)
iris_test = iris_test[(iris_test["target"] == 0) | (iris_test["target"] == 1)]


In [56]:
lr_model = LogisticRegression(data=iris_train)


cost,parameters = lr_model.iterations(n=100).get(99)
bias = parameters[0]
parameters = parameters[1:]
print(cost)
print(parameters)
print(bias)

0.009264700928096586
[-0.4950853  -1.91683721  2.88470107  1.24578537]
-0.31274558356913407


In [57]:
iris_test["linear_combination"] = iris_test.iloc[:,:-1].apply(lambda x: np.dot(x,parameters)+ bias,axis=1)
iris_test["predicted"] = iris_test["linear_combination"].apply(lambda x: (1 / (1 + np.exp(-x))))

In [71]:
iris_test["predicted"] = iris_test["predicted"].apply(lambda x: round(x))
iris_test

,sepal length (cm),sepal width (cm),petal length (cm),petal width (cm),target,linear_combination,predicted
97,6.2,2.9,4.3,1.3,1,5.082633,1
54,6.5,2.8,4.6,1.5,1,6.240359,1
3,4.6,3.1,1.5,0.2,0,-3.956125,0
75,6.6,3.0,4.4,1.4,1,5.105964,1
91,6.1,3.0,4.6,1.4,1,5.930447,1
79,5.7,2.6,3.5,1.0,1,3.223731,1
11,4.8,3.4,1.6,0.2,0,-4.341723,0
26,5.0,3.4,1.6,0.4,0,-4.191583,0
0,5.1,3.5,1.4,0.2,0,-5.258872,0
19,5.1,3.8,1.5,0.3,0,-5.420875,0


In [75]:
miscasses = iris_test[iris_test["target"] != iris_test["predicted"]]

miscasses

,sepal length (cm),sepal width (cm),petal length (cm),petal width (cm),target,linear_combination,predicted
